**Name:** VICENTE, Francheska Josefa

**Section:** DATAPRE N01

# Data Collection

In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Bulbapedia
Get the list of the next generations of Pokemons. Make sure you get the following information: (1) kdex, (2) ndex, (3) name, (4) types, and (5) URL to the Pokemon's wiki page.

In [33]:
URL = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"

page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
poke_content = soup.find(id='mw-content-text')
poke_tables = poke_content.find_all('table')

print(poke_tables)

[<table align="right" cellspacing="2" style="border: 2px solid #80964B; border-radius: 20px; -moz-border-radius: 20px; -webkit-border-radius: 20px; -khtml-border-radius: 20px; -icab-border-radius: 20px; -o-border-radius: 20px; margin-left: 5px; margin-bottom: 5px;" width="10%">
<tbody><tr>
<td align="center"><b>Shortcuts</b><br/><a class="mw-redirect" href="/wiki/Ndex" title="Ndex">Ndex</a><br/><a class="mw-redirect" href="/wiki/Olddex" title="Olddex">Olddex</a><br/><a class="mw-redirect" href="/wiki/Natdex" title="Natdex">Natdex</a>
</td></tr></tbody></table>, <table align="center" style="border-radius: 10px; -moz-border-radius: 10px; -webkit-border-radius: 10px; -khtml-border-radius: 10px; -icab-border-radius: 10px; -o-border-radius: 10px;; border: 2px solid #FF1111; background: #FF1111;">
<tbody><tr>
<th style="border-top-left-radius: 5px; -moz-border-radius-topleft: 5px; -webkit-border-top-left-radius: 5px; -khtml-border-top-left-radius: 5px; -icab-border-top-left-radius: 5px; -o-b

In [26]:
# source: 02 - Web Scraping With Edits from class
# edited to include the link of Pokemon
URL = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"

page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
poke_content = soup.find(id='mw-content-text')
poke_tables = poke_content.find_all('table')

min_generation = 2
max_generation = 9 # there are currently 9 generations of Pokemons

extracted_poke_info = []

for i in range (min_generation, max_generation + 1):
    info_start = 1
    gen_list = poke_tables[i]
    poke_info= gen_list.contents[1]
    info_row = gen_list.contents[info_start]
    
    for pokemon_info_values, even_index_chec in zip(info_row.contents, range(0,len(info_row.contents))):
        if ((even_index_chec % 2) == 0) & (even_index_chec != 0) :
            url = 'https://bulbapedia.bulbagarden.net' + info_row.contents[even_index_chec].find_all('a')[0]['href']
            pokemon_raw_info = pokemon_info_values.text.strip().split('\n')
            generation = i
            if len(pokemon_raw_info) == 7:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                poke_name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = ''
            elif len(pokemon_raw_info) == 8:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[1]
                poke_name = pokemon_raw_info[4]
                type1 = pokemon_raw_info[6]
                type2 = pokemon_raw_info[7]
            elif len(pokemon_raw_info) == 9:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                poke_name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = ''
            elif len(pokemon_raw_info) == 10:
                kdex = pokemon_raw_info[0]
                ndex = pokemon_raw_info[2]
                poke_name = pokemon_raw_info[6]
                type1 = pokemon_raw_info[8]
                type2 = pokemon_raw_info[9]
            else:
                print('Check out elements containing ' + str(len(pokemon_raw_info)) + ' elements')
        
            extracted_poke_info.append((kdex, ndex, poke_name, type1, type2, generation, url))
            
        else:
            pass

print(extracted_poke_info)

[('#001', '#152', 'Chikorita', 'Grass', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Chikorita_(Pok%C3%A9mon)'), ('#002', '#153', 'Bayleef', 'Grass', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Bayleef_(Pok%C3%A9mon)'), ('#003', '#154', 'Meganium', 'Grass', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Meganium_(Pok%C3%A9mon)'), ('#004', '#155', 'Cyndaquil', 'Fire', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Cyndaquil_(Pok%C3%A9mon)'), ('#005', '#156', 'Quilava', 'Fire', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Quilava_(Pok%C3%A9mon)'), ('#006', '#157', 'Typhlosion', 'Fire', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Typhlosion_(Pok%C3%A9mon)'), ('#157', '', 'Typhlosion', 'Fire', 'Ghost', 2, 'https://bulbapedia.bulbagarden.net/wiki/Typhlosion_(Pok%C3%A9mon)'), ('#007', '#158', 'Totodile', 'Water', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Totodile_(Pok%C3%A9mon)'), ('#008', '#159', 'Croconaw', 'Water', '', 2, 'https://bulbapedia.bulbagarden.net/wiki/Croc

## News site
Get the news articles published on April 4 - 5. Decide which news site you want to get articles from. Make sure you get the following information: (1) date, (2) title, (3) full article, and (4) author.

In [141]:
from datetime import datetime

start = 'april 4, 2022'
end = 'april 5, 2022'

START_DATE = datetime.strptime(start, "%B %d, %Y")
END_DATE = datetime.strptime(end, "%B %d, %Y")

checker = True
counter = 1

extracted_articles = []

while checker:
    if counter == 1:
        URL = "https://newsinfo.inquirer.net/source/philippine-daily-inquirer"
    else:
        URL = "https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/" + str(counter)
    
    page = requests.get(URL)
    print(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    news_content = soup.find(id='inq-channel-left')
    articles_list = news_content.find_all(id="ch-ls-box")
    
    for i in range (len(articles_list)): 
        article = articles_list[i]
        article_text = article.text.strip().split('\n')
        
        
        if len(article_text) >= 3:
            date_str = article_text[2].split(' BY:')[0]
            curr_date = datetime.strptime(date_str, "%B %d, %Y")
            
            article_url = article.find_all('a')[0]['href']
            article_page = requests.get(article_url)
            article_soup = BeautifulSoup(article_page.content, 'html.parser')

            article_title = article_text[1]
            article_content = article_soup.find(id="article_content")
            
            if(type(article_soup.find(id="art_author")) != 'NoneType'):
                article_author = article_soup.find(id="art_author").find_all("a")[0].text.strip()
            else:
                article_author = ''

            extracted_articles.append((article_title, article_author, date_str, article_content))

            
            if curr_date < START_DATE:
                checker = False
            elif START_DATE <= curr_date <= END_DATE:
                article_url = article.find_all('a')[0]['href']
                article_page = requests.get(article_url)
                article_soup = BeautifulSoup(article_page.content, 'html.parser')

                article_title = article_text[1]
                article_content = article_soup.find(id="article_content")
                article_author = article_soup.find(id="art_author").find_all("a")[0].text.strip()

                extracted_articles.append((article_title, article_author, date_str, article_content))
                
            else:
                continue
    if counter == 3:
        checker = False
    counter = counter + 1
    
print(extracted_articles)

https://newsinfo.inquirer.net/source/philippine-daily-inquirer
https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/2
https://newsinfo.inquirer.net/source/philippine-daily-inquirer/page/3


AttributeError: 'NoneType' object has no attribute 'find_all'

In [84]:
import datetime as dt
start = "2022-04-04"
end   = "2022-04-05"


URL = "https://newsinfo.inquirer.net/source/philippine-daily-inquirer"

page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
news_content = soup.find(id='inq-channel-left')
articles_list = news_content.find_all(id="ch-ls-box")

extracted_articles = []

articles_list[2].text.strip().split('\n')

['Headlines',
 'Amid push for boosters, many in PH still fear ‘side effects’',
 'April 14, 2022 BY:\xa0 Kathleen de Villa']

In [65]:
dates = 'june 1, 2005'


date = datetime.strptime(dates, "%B %d, %Y")
print(date)

2005-06-01 00:00:00


In [69]:
from datetime import datetime

start = 'april 4, 2022'
end = 'april 5, 2022'

START_DATE = datetime.strptime(start, "%B %d, %Y")
END_DATE = datetime.strptime(end, "%B %d, %Y")

curr_date = datetime.strptime('april 5, 2022', "%B %d, %Y")

if (START_DATE <= curr_date <= END_DATE):
    print("heyyo")

heyyo
